# Test solver harmonic

In [1]:
import os, sys, time
import argparse
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt
import scipy
import healpy as hp
import astropy.io.fits as fits
import camb
import jax
import jax.numpy as jnp
import jax.scipy as jsp
import jax_healpy as jhp
import jax.random as random
import toml
import numpyro
from functools import partial
import micmac
from micmac import *
from fgbuster.observation_helpers import *

In [2]:
from jax import config
current_path = os.path.abspath('')
sys.path.append(current_path)

config.update("jax_enable_x64", True)

In [3]:
path_toml_file = current_path+'/Harm_fullsky_v1a.toml'
path_home_test_playground = os.path.dirname(current_path) + '/'

In [4]:
MICMAC_obj = create_Harmonic_MICMAC_sampler_from_toml_file(path_toml_file)


In [5]:
seed_realization_input = MICMAC_obj.seed

In [6]:
fgs_model_ = 'd0s0'
instr_name = MICMAC_obj.instrument_name #'SO_SAT'

path_Fisher = path_home_test_playground + f'Fisher_matrix_{MICMAC_obj.instrument_name}_EB_model_{fgs_model_}_noise_True_seed_42_lmin2_lmax128.txt'
try :
    Fisher_matrix = np.loadtxt(path_Fisher)
except:
    print("Fisher matrix not found !", flush=True)
    Fisher_matrix = np.loadtxt(path_home_test_playground + f'Fisher_matrix_{MICMAC_obj.instrument_name}_EB_model_d0s0_noise_True_seed_42_lmin2_lmax128.txt')


In [7]:
instrument = get_instrument(instr_name)


In [8]:
np.random.seed(seed_realization_input)
freq_maps_fgs_noised = get_observation(instrument, fgs_model_, nside=MICMAC_obj.nside, noise=True)[:, 1:, :]   # keep only Q and U
np.random.seed(seed_realization_input)
freq_maps_fgs_denoised = get_observation(instrument, fgs_model_, nside=MICMAC_obj.nside, noise=False)[:, 1:, :]   # keep only Q and U

noise_map = freq_maps_fgs_noised - freq_maps_fgs_denoised
 

freq_maps_fgs = freq_maps_fgs_denoised + noise_map


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [9]:
# Mixing matrix initialization
init_mixing_matrix_obj = micmac.InitMixingMatrix(np.array(instrument['frequency']), MICMAC_obj.n_components, pos_special_freqs=[0,-1])

init_params = init_mixing_matrix_obj.init_params()

mixing_matrix_obj = micmac.MixingMatrix(instrument['frequency'], MICMAC_obj.n_components, init_params, pos_special_freqs=[0,-1])
mixing_matrix_sampled = mixing_matrix_obj.get_B()



In [10]:
# frequency_inv_noise_cell = get_true_Cl_noise(instrument['depth_p'], MICMAC_obj.lmax)

# MICMAC_obj.freq_noise_c_ell = frequency_inv_noise_cell

In [11]:
covariance_matrix = np.linalg.inv(Fisher_matrix)

MICMAC_obj.covariance_step_size_B_f = covariance_matrix[:-1, :-1]
MICMAC_obj.step_size_r = np.sqrt(covariance_matrix[-1, -1])

In [12]:
input_freq_maps, input_cmb_maps, theoretical_red_cov_r0_total, theoretical_red_cov_r1_tensor = MICMAC_obj.generate_input_freq_maps_from_fgs(freq_maps_fgs, return_only_freq_maps=False)


Calculating spectra from CAMB !
Calculating spectra from CAMB !


In [13]:
theoretical_r0_total = get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r0_total)
theoretical_r1_tensor = get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r1_tensor)

c_ell_approx = np.zeros((3,MICMAC_obj.lmax+1))
c_ell_approx[0,MICMAC_obj.lmin:] = theoretical_r0_total[0,:]
c_ell_approx[1,MICMAC_obj.lmin:] = theoretical_r0_total[1,:]

red_cov_approx_matrix = get_reduced_matrix_from_c_ell_jax(c_ell_approx)[MICMAC_obj.lmin:]

In [14]:
len_pos_special_freqs = len(MICMAC_obj.pos_special_freqs)

## Starting MH

In [15]:
minimum_std_Fisher_diag = jnp.diag(scipy.linalg.sqrtm(jnp.linalg.inv(Fisher_matrix)))
sigma_gap = 20

dimension_free_param_B_f = 2*(MICMAC_obj.n_frequencies-len_pos_special_freqs)

In [16]:
minimum_std_Fisher_diag

Array([1.22719235e-04, 4.72984090e-05, 4.44084763e-05, 5.23546882e-05,
       4.28340683e-05, 1.62270766e-05, 1.52905384e-05, 1.82529746e-05,
       5.55036342e-04], dtype=float64)

In [17]:
np.random.seed(MICMAC_obj.seed)
first_guess = jnp.zeros(dimension_free_param_B_f)
first_guess = first_guess.at[MICMAC_obj.indexes_free_Bf].set(
    init_params.ravel(order='F')[MICMAC_obj.indexes_free_Bf] + minimum_std_Fisher_diag[:-1]*np.random.uniform(low=-sigma_gap,high=sigma_gap, size=(dimension_free_param_B_f)))
init_params_mixing_matrix = first_guess.reshape((MICMAC_obj.n_frequencies-len_pos_special_freqs),2,order='F')

first_guess_r = np.random.uniform(low=-sigma_gap,high=sigma_gap, size=(1,))*minimum_std_Fisher_diag[-1] + MICMAC_obj.r_true

In [18]:
print("Difference first params", np.abs(init_params - first_guess.reshape((MICMAC_obj.n_frequencies-len_pos_special_freqs),2,order='F')),
                                    np.abs(first_guess_r - MICMAC_obj.r_true))
print("True values", first_guess_r, MICMAC_obj.r_true)

Difference first params [[0.00061585 0.00058936]
 [0.00085272 0.00022329]
 [0.0004121  0.00027029]
 [0.00020661 0.00026735]] [0.0022449]
True values [0.0122449] 0.01


In [19]:
bounds = [(None, None)]*(dimension_free_param_B_f)
bounds.append((-0.5, 10.0))

options_minimizer_TNC = {'options':
                            {'maxCGit':-1, 
                            'eta': 0.25, 
                            'stepmx': 0, 
                            'accuracy': 0, 
                            'minfev': 0, 
                            'ftol': - 1, 
                            'xtol': -1, 
                            'gtol': -1, 
                            'rescale': - 1, 
                            'disp': False, 
                            'finite_diff_rel_step': None, 
                            'maxfun':10000},
                        'method':'TNC',
                        }


options_minimizer_BFGS = {'maxiter':500, 
                            'tol':0.001, 
                            'stepsize':0.0, 
                            'linesearch':'zoom', 
                            'linesearch_init':'increase', 
                            'stop_if_linesearch_fails':False, 
                            'condition':None, 
                            'maxls':15, 
                            'decrease_factor':None, 
                            'increase_factor':1.5, 
                            'max_stepsize':1.0, 
                            'min_stepsize':1e-06, 
                            'history_size':10, 
                            'use_gamma':True, 
                            'implicit_diff':True, 
                            'implicit_diff_solve':None, 
                            'jit':'auto', 
                            'unroll':'auto', 
                            'verbose':False}

In [20]:
params_final = MICMAC_obj.perform_harmonic_minimize(input_freq_maps, c_ell_approx, init_params_mixing_matrix,
                                    theoretical_r0_total=theoretical_r0_total, theoretical_r1_tensor=theoretical_r1_tensor,
                                    initial_guess_r=first_guess_r,
                                    method_used='ScipyMinimize',
                                    **options_minimizer_TNC)


Start of minimization
End of minimization
Found parameters [0.33857941 0.0296126  0.00997524 0.00328001 0.00973012 0.0514598
 0.12913631 0.44078292 0.00913313]
With state ScipyMinimizeInfo(fun_val=Array(-2.32528048e+09, dtype=float64, weak_type=True), success=False, status=4, iter_num=23, hess_inv=None, num_fun_eval=Array(144, dtype=int32), num_jac_eval=Array(0, dtype=int32), num_hess_eval=Array(0, dtype=int32))


In [22]:
print("True B_f parameters : ", init_params)
print("Estimated B_f parameters : ", params_final[:-1])
print("Difference in B_f : ", params_final[:-1].reshape((MICMAC_obj.n_frequencies-len_pos_special_freqs,2),order='F') - init_params)

print("##############################################")
print("True r parameter : ", MICMAC_obj.r_true)
print("Estimated r parameter : ", params_final[-1])
print("Difference in r : ", params_final[-1] - MICMAC_obj.r_true)

print("##############################################")
print("Relative errors B_f :", np.abs(params_final[:-1].reshape((MICMAC_obj.n_frequencies-len_pos_special_freqs,2),order='F') - init_params)/(init_params))
print("Relative errors r :", np.abs(params_final[-1] - MICMAC_obj.r_true)/MICMAC_obj.r_true)

True B_f parameters :  [[0.33858815 0.00979233]
 [0.02963068 0.05147511]
 [0.00998213 0.12913911]
 [0.00328725 0.44076761]]
Estimated B_f parameters :  [0.33857941 0.0296126  0.00997524 0.00328001 0.00973012 0.0514598
 0.12913631 0.44078292]
Difference in B_f :  [[-8.73258524e-06 -6.22163394e-05]
 [-1.80825432e-05 -1.53133842e-05]
 [-6.88334716e-06 -2.80657441e-06]
 [-7.24136134e-06  1.53103125e-05]]
##############################################
True r parameter :  0.01
Estimated r parameter :  0.009133131546562012
Difference in r :  -0.0008668684534379884
##############################################
Relative errors B_f : [[2.57911724e-05 6.35357546e-03]
 [6.10264182e-04 2.97491025e-04]
 [6.89567160e-04 2.17329541e-05]
 [2.20286010e-03 3.47355659e-05]]
Relative errors r : 0.08668684534379884
